In [14]:
# create studentroll-name-allocation csv file-mappings.csv

#from f1.csv
import pandas as pd

# Define the input and output file names
input1 = "f1.csv"
input2 = "f2.csv"
input3 = "f3.csv"
input4 = "f4.csv"
input5 = "f5.csv"
output_filename = "mappings.csv"


# Read the input CSV file into a pandas DataFrame
input_df1 = pd.read_csv(input1)
input_df2 = pd.read_csv(input2)
input_df3 = pd.read_csv(input3)
input_df4 = pd.read_csv(input4)
input_df5 = pd.read_csv(input5)


# Extract specific columns (Roll No, Name of Student, Allocation)
columns_to_extract = ["Roll No", "Name of Student", "Allocation"]
output_df1 = input_df1[columns_to_extract]
output_df2 = input_df2[columns_to_extract]
output_df3 = input_df3[columns_to_extract]
output_df4 = input_df4[input_df4["Eligible for TAship"] == "Yes"][columns_to_extract]
output_df5 = input_df5[input_df5["Eligible for TAship"] == "Yes"][columns_to_extract]


# Write the extracted data to the output CSV file
output_df1.to_csv(output_filename, index=False)
output_df2.to_csv(output_filename, mode='a', header=False, index=False)
output_df3.to_csv(output_filename, mode='a', header=False, index=False)
output_df4.to_csv(output_filename, mode='a', header=False, index=False)
output_df5.to_csv(output_filename, mode='a', header=False, index=False)







In [15]:
import pandas as pd
from fuzzywuzzy import fuzz

# Function to calculate the percent match between two names
def name_similarity(name1, name2):
    return fuzz.ratio(name1, name2)

# Read the first CSV file
df1 = pd.read_csv('mappings.csv')

# Create a mapping of names to Roll No from the first CSV file
name_to_roll = {row['Name of Student']: row['Roll No'] for index, row in df1.iterrows()}



# Read the second CSV file
df2 = pd.read_csv('courses_info_name.csv')
count=0
# # Process the data from the second CSV file
result_data = []
for index, row in df2.iterrows():
    course = row['Course']
    ta = row['TA']
    names = row['Names']
    matching_roll_no = None
    if not pd.isna(names) and names != "None":
        names = names.split('-')
        count+=1
        for name in names:
            name = name.strip()  # Remove leading/trailing spaces
            best_match_score=0
            best_match_name=None
            for mapped_name in name_to_roll:
                similarity_score = name_similarity(name, mapped_name)
                if similarity_score > best_match_score:
                    best_match_name = mapped_name
                    best_match_score = similarity_score
            if best_match_name is not None:
                matching_roll_no = name_to_roll[best_match_name]
            result_data.append([course,ta,matching_roll_no])


# Create a DataFrame for the result
result_df = pd.DataFrame(result_data, columns=['Course', 'TA', 'Roll No'])
print(count)
# Write the result to a third CSV file
result_df.to_csv('course_info_roll.csv', index=False)


25


In [49]:
## Verification of Code Maximum Matching
import numpy as np
from my_matching import *
import networkx as nx
n=3
matrix = np.random.randint(50, 91, (n, n))
np.fill_diagonal(matrix, 100)
# for col in range(n):
#     np.random.shuffle(matrix[:, col])
matrix_dict = {}
vacancies={}
for row_number, row_values in enumerate(matrix):
    row_dict = {f'S{col+1}': value for col, value in enumerate(row_values)}
    matrix_dict[f'C{row_number+1}'] = row_dict
    vacancies[f'C{row_number+1}']=1

match,max_weight=matching(matrix_dict,vacancies)
print(match)
print(max_weight)
# print(vacancies)


{('S1', 'C1_0'), ('S2', 'C2_0'), ('S3', 'C3_0')}
300


In [16]:
df = pd.read_csv('course_info_roll.csv')

# Create a dictionary with course names as keys and lists of roll numbers as values
course_imp = {}
for index, row in df.iterrows():
    course = row['Course']
    roll_no = row['Roll No']
    
    if course in course_imp:
        course_imp[course].append(roll_no)
    else:
        course_imp[course] = [roll_no]
df1= pd.read_csv('courses_info_name.csv')
for index, row in df1.iterrows():
    course=row['Course']
    if course not in course_imp:
        course_imp[course]=[]

print(course_imp['DeptWork'])

[]


In [50]:
import random
df1=pd.read_csv('mappings.csv')
df2=pd.read_csv('courses_info_name.csv')
rolls=list(df1['Roll No'])
courses=list(df2['Course'])
vacancies=list(df2['TA'])
d1={}
d2={}
for i in range(len(courses)):
    d1[courses[i]]=vacancies[i]
for c in courses:
    temp={}
    count=d1[c]*100
    m=0
    for j in rolls:
        temp[j]=0
        if j in course_imp[c]:
            m+=1
    n=d1[c]-m
    if n<0:
        for k in course_imp[c]:
            temp[k]=count/m
    elif n==0 or m==0:
        for k in rolls:
            temp[k]=100
    else:
        a=100-20*(m/n)
        if a<80:
            a=80
        # print(m,n,a,c)
        random_number = random.randint(int(a), 100)
        for k in rolls:
            if k in course_imp[c]:
                temp[k]=100+(100-random_number)*(n/m)
            else:
                temp[k]=random_number
    d2[c]=temp



In [51]:
from my_matching import *
m,w=matching(d2,d1)
print(m)


{('22M0806', 'CS101_41'), ('22M0795', 'CS101_32'), ('23M0797', 'CS293_12'), ('204050004', 'SysadTA_1'), ('22m0742', 'CS725_1'), ('CS747_0', '23M0773'), ('DeptWork_0', '21Q050009'), ('194053001', 'CSEWebteam_0'), ('23M2105', 'CS101_58'), ('22m0758', 'CS347_11'), ('23M0755', 'CS293_4'), ('23M0805', 'CS230_5'), ('22m0809', 'CS101_40'), ('23M0794', 'CS293_3'), ('194052001', 'DeptWork_1'), ('22m2107', 'CS101_21'), ('23M0766', 'CS232_11'), ('23M0796', 'CS154_11'), ('22M0741', 'CS101_30'), ('23M0764', 'CS475_1'), ('21Q050010', 'CS101_13'), ('22m0767', 'CS635_0'), ('23M0747', 'CS293_5'), ('23M0770', 'CS232_8'), ('23M0795', 'CS207_7'), ('22M0798', 'CS781_0'), ('23M0812', 'CS230_0'), ('194050014', 'CS105_3'), ('214050010', 'CS152_3'), ('23D0366', 'CS154_2'), ('22M0769', 'CS101_24'), ('23M2109', 'CS101_54'), ('22M2101', 'CS337_2'), ('22D0364', 'CS105_4'), ('22M0816', 'CS101_36'), ('23M0821', 'CS218M_1'), ('23M2106', 'CS101_57'), ('23D0364', 'CS154_4'), ('22M0814', 'CS101_37'), ('23M2103', 'CS101_

In [53]:
l=[]
for t in m:
    if t[0][0]!='1' and t[0][0]!='2':
        l.append([t[0],t[1]])
    else:
        l.append([t[1],t[0]])


# Remove the part after the underscore in course names
data = [[course.split('_')[0], course_code] for course, course_code in l]

# Create a DataFrame from the modified data
df = pd.DataFrame(data, columns=['Course', 'Roll No'])

# Write the DataFrame to a CSV file
csv_file = 'output.csv'
df.to_csv(csv_file, index=False)
        

In [64]:

import pandas as pd
df1=pd.read_csv("course_info_roll.csv")
df2=pd.read_csv("output.csv")
df3=pd.read_csv("mappings.csv")
courses_list=list(set(df1["Course"]))
rolls=list(df1['Roll No'])
total_data=[]
requests=0
alloted_before=0
alloted_matching=0
for c in courses_list:
    count1=0
    count2=0
    for r in course_imp[c]:
        if df1[df1['Roll No']==r].iloc[0,0] == df2[df2['Roll No'] == r].iloc[0,0]:
            count1+=1
        if df1[df1['Roll No']==r].iloc[0,0] == df3[df3['Roll No'] == r].iloc[0,2]:
            count2+=1
    requests+=len(course_imp[c])
    alloted_before+=count2
    alloted_matching+=count1
    total_data.append([c,len(course_imp[c]),count2,count1])
    
    # print(f"Course name:{c} Requests:{len(course_imp[c])} Originally Allocated:{count2} Allocated By Matching:{count1}\n")

result_df = pd.DataFrame(total_data, columns=['Course', 'No of Requests', 'Originally Allocated last year','Allocated by Maxium Matching'])

result_df.to_csv('analysis.csv', index=False)
print(requests)
print(alloted_before)
print(alloted_matching)


118
60
64
